# Generating sequences on the fly on submission

In [1]:
import numpy as np
import datetime
import time
import edd.experiments as edde
from edd.backend import IBMQBackend
from edd.circuit import IBMQDdCircuit
from edd.pulse import IBMQDdSchedule
from edd.data import IBMQData
import edd.workflow as wf

from matplotlib import rc
rc("text", usetex=False)

# Submission Settings

In [55]:
# states to sample
offset = 0
# use symmetric pulses or not
sym = False
# delay between pulses
delay = 0
# label that describes the type of pulse delay experiment going on
d_label = "min"
# basis for pulses
basis = 'g_basis'
# qubits on which to encode and to apply DD on
qubit = 0
encoding_qubits = [qubit]
dd_qubits = [qubit]

sched_settings = [offset, sym, delay, d_label, basis, encoding_qubits, dd_qubits]

backend_name = 'armonk'
# deprecated way to load in backend when my account info was hard-coded
#backend = IBMQBackend(f'ibmq_{backend_name}', "Nic")
# use this with your own info...
backend = IBMQBackend(name, hub, group, project, token) 
max_time = 150 # in micro seconds
sample_points = 12

ibmqfactory.load_account:WARNING:2022-04-12 23:06:54,395: Credentials are already in use. The existing account in the session will be replaced.


IBM Backend ibmq_jakarta loaded at date/time: 2022-04-12 23:06:58


In [56]:
backend.backend.remaining_jobs_count()

20

In [57]:
sched = IBMQDdSchedule(backend, "g_basis")
sched.add_x(0)
drawn = sched.draw()

In [58]:
sched.get_phys_time()

35.55555555555556

In [60]:
backend.

# Create rep dict for sequences of interest

In [5]:
basis = 'g_basis'
sched_settings = [offset, sym, delay, d_label, basis, encoding_qubits, dd_qubits]
list_of_seq = ['purex']

np.random.shuffle(list_of_seq)

    
# make repetitions dictionary for sequences
rep_dict = {seq: [] for seq in list_of_seq}
for seq in list_of_seq:
    sched = IBMQDdSchedule(backend, basis)
    if "uddx" in seq:
        seq_method = "add_uddx"
        ord_n = int(seq[4:])
        getattr(sched, seq_method)(ord_n, qubit, 1, delay, sym)
    elif "qdd" in seq:
        seq_method = "add_qdd"
        _, ord_n, ord_m = seq.split("_")
        ord_n = int(ord_n)
        ord_m = int(ord_m)
        getattr(sched, seq_method)(ord_n, ord_m, qubit, 1, delay, sym)
    elif "ur" in seq and "x" not in seq and "y" not in seq:
        seq_method = "add_ur"
        ord_n = int(seq[2:])
        getattr(sched, seq_method)(ord_n, qubit, 1, delay, sym)
    else:
        getattr(sched, f"add_{seq}")(qubit, 1, delay, sym)
    T = sched.get_phys_time() / 1000
    max_reps = np.floor(max_time / T)
    rep_list = sorted(list(set(map(int, np.linspace(0, max_reps, sample_points)))))
    rep_dict[seq] = rep_list

In [7]:
sched.reset()
sched.add_x(0)
sched.get_duration()

320

In [11]:
wf.gen_fds_on_fly_and_submit(backend, rep_dict, sched_settings, 15)

job idx being tried: 0 w/ seq purey
avail jobs: 20
submitting job with idx: 0 w/ seq purey
job with idx: 0 failed on first try w/ seq purey
job idx being tried: 1 w/ seq purey
avail jobs: 20
submitting job with idx: 1 w/ seq purey
job with idx: 1 failed on first try w/ seq purey
job idx being tried: 2 w/ seq purey
avail jobs: 20
submitting job with idx: 2 w/ seq purey
job with idx: 2 failed on first try w/ seq purey
job idx being tried: 3 w/ seq purey
avail jobs: 20
submitting job with idx: 3 w/ seq purey
job with idx: 3 failed on first try w/ seq purey
job idx being tried: 4 w/ seq purey
avail jobs: 20
submitting job with idx: 4 w/ seq purey
job with idx: 4 failed on first try w/ seq purey
job idx being tried: 5 w/ seq purey
avail jobs: 20
submitting job with idx: 5 w/ seq purey
job with idx: 5 failed on first try w/ seq purey
job idx being tried: 6 w/ seq purey
avail jobs: 20
submitting job with idx: 6 w/ seq purey
job with idx: 6 failed on first try w/ seq purey
job idx being tried:

KeyboardInterrupt: 

## Cancelling a job (just in case)

In [4]:
backend.backend.remaining_jobs_count()

7

In [5]:
for j in range(13):
    backend.backend.active_jobs()[0].cancel()